Common Spatial Patterns

In [ ]:
class CommonSpatialPatterns:
    def __init__(self, n_components=8):
        self.n_components = n_components
        self.filters = None
        
    def fit(self, X, y):
        """Fit CSP filters to the data"""
        # Compute covariance matrices for each class
        classes = np.unique(y)
        n_classes = len(classes)
        
        # Compute average covariance matrix for each class
        covs = []
        for class_label in classes:
            class_data = X[y == class_label]
            cov_matrix = np.zeros((X.shape[1], X.shape[1]))
            for trial in class_data:
                cov_matrix += np.cov(trial)
            cov_matrix /= len(class_data)
            covs.append(cov_matrix)
        
        # Solve generalized eigenvalue problem
        eigenvals, eigenvecs = scipy.linalg.eigh(covs[0], covs[0] + covs[1])
        
        # Sort by eigenvalue
        idx = np.argsort(eigenvals)[::-1]
        eigenvals = eigenvals[idx]
        eigenvecs = eigenvecs[:, idx]
        
        # Select top and bottom components
        n_top = self.n_components // 2
        self.filters = np.concatenate([eigenvecs[:, :n_top], eigenvecs[:, -n_top:]], axis=1)
        
        return self
    
    def transform(self, X):
        """Apply CSP filters to data"""
        if self.filters is None:
            raise ValueError("CSP must be fitted before transform")
        
        X_transformed = np.zeros((X.shape[0], self.filters.shape[1], X.shape[2]))
        for i, trial in enumerate(X):
            X_transformed[i] = self.filters.T @ trial
        
        return X_transformed

In [ ]:
# CSP and normalization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Normalizing Labels to [0, 1, 2, 3]
y_train = first_session_labels - np.min(first_session_labels)
y_test = second_session_labels - np.min(second_session_labels)

print(f"First session data shape: {first_session_data.shape}")
print(f"Second session data shape: {second_session_data.shape}")

# Apply CSP spatial filtering
csp = CommonSpatialPatterns(n_components=16)
X_first_csp = csp.fit(first_session_data, y_train).transform(first_session_data)

# Transform second session data using the same CSP filters
X_second_csp = csp.transform(second_session_data)

# normalization - robust z-score per channel and trial
def robust_normalize(data):
    """Robust normalization using median and MAD"""
    normalized_data = np.zeros_like(data)
    for trial in range(data.shape[0]):
        for channel in range(data.shape[1]):
            channel_data = data[trial, channel, :]
            median = np.median(channel_data)
            mad = np.median(np.abs(channel_data - median))
            if mad > 0:
                normalized_data[trial, channel, :] = (channel_data - median) / (1.4826 * mad)
            else:
                normalized_data[trial, channel, :] = channel_data - median
    return normalized_data

# Apply robust normalization
X_first_session = robust_normalize(X_first_csp)
X_second_session = robust_normalize(X_second_csp)